In [ ]:
import PyPDF2
import re
import os
import gradio as gr
import webbrowser
from threading import Timer

# --- Configuración ---
PDF_PATH = "Prueba2_SP.pdf"
MIN_MATCH_SCORE = 2 # Requiere al menos 2 palabras clave
PORT = 7862 # Puerto para el servidor web

# --- Lógica del Chatbot (Funciones de la Parte 1) ---

def extract_text_from_pdf(pdf_path):
    """Extrae texto de cada página de un archivo PDF."""
    pages_text = []
    if not os.path.exists(pdf_path):
        print(f"Error: El archivo PDF '{pdf_path}' no fue encontrado.")
        # Lanza una excepción para que Gradio muestre un error claro
        raise FileNotFoundError(f"Error crítico: No se encontró el archivo '{pdf_path}'. Asegúrate de que está en el directorio correcto.")
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"Leyendo {num_pages} páginas del PDF...")
            for page_num in range(num_pages):
                try:
                    page = reader.pages[page_num]
                    text = page.extract_text()
                    pages_text.append(text if text else "") # Añade string vacío si no extrae nada
                except Exception as page_e:
                    print(f"Advertencia: No se pudo extraer texto de la página {page_num + 1}. Error: {page_e}")
                    pages_text.append("") # Añade página vacía si hay error en una página específica
            print("Extracción de texto completada.")
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
        # Lanza una excepción para detener la app si la carga falla
        raise RuntimeError(f"Error al procesar el PDF: {e}")
    return pages_text

def preprocess_text(text):
    """Limpia y normaliza el texto."""
    if not isinstance(text, str): # Asegura que la entrada sea string
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\sáéíóúüñ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def find_relevant_page(query, pages_text):
    """Encuentra la página más relevante basado en palabras clave."""
    processed_query = preprocess_text(query)
    query_words = set(processed_query.split())

    if not query_words:
        return "Por favor, introduce una consulta válida."

    best_page_index = -1
    best_score = 0

    for i, page_content in enumerate(pages_text):
        processed_page = preprocess_text(page_content)
        if not processed_page: # Saltar páginas vacías o con error de extracción
             continue
        page_words = set(processed_page.split())
        current_score = len(query_words.intersection(page_words))

        important_keywords = {"vacaciones", "licencia", "enfermedad", "salario", "despido", "ética", "confidencialidad", "horario", "jurado", "testigo", "seguridad"}
        # Bonus más significativo por palabras clave importantes
        bonus_score = sum(1 for keyword in important_keywords if keyword in query_words and keyword in page_words)
        current_score += bonus_score * 2 # Dar más peso a las palabras clave

        if current_score > best_score:
            best_score = current_score
            best_page_index = i

    if best_score >= MIN_MATCH_SCORE and best_page_index != -1:
        page_number = best_page_index + 1
        # Formatea un poco mejor la respuesta para el chat
        response = (
            f"**Información encontrada (Página {page_number} del Manual):**\n\n"
            f"_{pages_text[best_page_index].strip()[:800]}..._" # Muestra un extracto más largo
            f"\n\n*(Nota: Esta es la página completa que parece más relevante ({best_score} puntos). Puede contener más detalles.)*"
        )
        return response
    else:
        return "Lo siento, no pude encontrar información específica sobre tu consulta en el Manual del Empleado. Intenta reformular tu pregunta con otras palabras clave o consulta directamente con Recursos Humanos."

# --- Carga del Manual (Hacerlo una sola vez al inicio) ---
try:
    print("Iniciando carga del Manual del Empleado...")
    manual_pages_content = extract_text_from_pdf(PDF_PATH)
    if not manual_pages_content:
         print("Advertencia: El manual parece estar vacío o no se pudo leer ninguna página.")
         # Permitir que la app corra, pero informará al usuario
         manual_pages_content = [] # Asegurar que sea una lista
except Exception as startup_e:
     print(f"Error fatal durante la carga inicial: {startup_e}")
     # Si falla la carga inicial, Gradio no debería iniciarse o mostrar un error persistente
     # En un entorno real, aquí podrías detener el script o configurar Gradio para mostrar un mensaje de error.
     # Por simplicidad aquí, imprimimos y continuamos; la función `respond` manejará la lista vacía.
     manual_pages_content = []


# --- Función de Interacción para Gradio ---
def respond(message):
    """
    Función que Gradio llamará para obtener la respuesta del bot.

    Args:
        message (str): El mensaje enviado por el usuario.

    Returns:
        str: La respuesta del chatbot.
    """
    if not manual_pages_content:
         return "Error: No se pudo cargar la información del Manual del Empleado. Por favor, contacta al administrador."

    print(f"Recibida consulta: {message}")
    bot_response = find_relevant_page(message, manual_pages_content)
    print(f"Respuesta generada: {bot_response[:100]}...") # Log corto de la respuesta
    return bot_response

# Función para abrir el navegador automáticamente
def open_browser():
    webbrowser.open_new(f"http://localhost:{PORT}")

# --- CSS personalizado para autoajustar el tamaño ---
custom_css = """
/* Estilo general para autoajuste de contenido */
.gradio-container {
    max-width: 100% !important;
    padding: 0 !important;
    margin: 0 auto !important;
}

/* Hacer que el chatbot se expanda según el contenido */
.chatbot-container {
    width: 100% !important;
    height: auto !important;
    min-height: 400px !important;
    overflow: auto !important;
}

/* Mejorar el ancho de los mensajes */
.message-wrap {
    width: 100% !important;
    max-width: none !important;
}

/* Ajustar el ancho de los bloques de código */
.message-wrap pre {
    white-space: pre-wrap !important;
    width: 100% !important;
    max-width: 100% !important;
    overflow-x: auto !important;
}

/* Mejorar la visualización de los mensajes del chat */
.message {
    padding: 15px !important;
    overflow-wrap: break-word !important;
    width: auto !important;
    max-width: 95% !important;
}
"""

# --- Creación de la Interfaz Gradio usando Blocks ---
print("Creando la interfaz Gradio...")

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="sky"), css=custom_css) as demo:
    gr.Markdown("# 💬 Chatbot RRHH - Asociación Pro Desarrollo Comunal del Patio, Inc.")
    gr.Markdown("Consulta información del Manual del Empleado. Escribe tu pregunta abajo.")
    
    with gr.Row():
        with gr.Column(scale=1):
            pass
        
        with gr.Column(scale=12):
            chatbot = gr.Chatbot(height="auto", elem_classes="chatbot-container")
            
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Escribe tu pregunta aquí (ej: '¿Cuántos días de vacaciones tengo?', 'Política sobre enfermedad')",
                    container=False,
                    scale=9
                )
                clear = gr.Button("Limpiar", scale=1)
            
            gr.Markdown("### Ejemplos de preguntas:")
            gr.Examples(
                examples=[
                    "¿Cuál es la política de vacaciones?",
                    "Información sobre licencia por enfermedad",
                    "¿Qué dice el código de ética?",
                    "Hostigamiento sexual",
                    "¿Hay bono de navidad?",
                    "Uso del teléfono personal",
                    "Periodo probatorio"
                ],
                inputs=msg
            )
        
        with gr.Column(scale=1):
            pass
    
    # Función para procesar mensaje del usuario y obtener respuesta
    def user_message(user_input, history):
        if not user_input:
            return gr.update(), history
        
        # Obtener respuesta del bot
        response = respond(user_input)
        
        # Añadir al historial
        history = history + [[user_input, response]]
        
        return "", history
    
    # Conectar eventos
    msg.submit(user_message, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: [], None, chatbot)

# --- Lanzar la aplicación ---
if __name__ == "__main__":
    print("Lanzando la aplicación Gradio... Se abrirá automáticamente en tu navegador.")
    # Abrir navegador con un pequeño retraso para dar tiempo al servidor
    Timer(1.5, open_browser).start()
    # Lanzar en modo "inbrowser=False" para que sea una ventana independiente
    demo.launch(server_port=PORT, share=True, inbrowser=False, pwa=True)